In [ ]:
!git clone https://github.com/pizadi/ISIC2018-training.git
from os import chdir
chdir('./ISIC2018-training')

In [ ]:
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Validation_Input.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Validation_GroundTruth.zip
# !wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Test_Input.zip

In [ ]:
!unzip ./ISIC2018_Task1-2_Training_Input.zip
!rm ./ISIC2018_Task1-2_Training_Input.zip
!unzip ./ISIC2018_Task1_Training_GroundTruth.zip
!rm ./ISIC2018_Task1_Training_GroundTruth.zip
!unzip ./ISIC2018_Task1-2_Validation_Input.zip
!rm ./ISIC2018_Task1-2_Validation_Input.zip
!unzip ./ISIC2018_Task1_Validation_GroundTruth.zip
!rm ./ISIC2018_Task1_Validation_GroundTruth.zip
!mkdir ./Preproc

In [1]:
import os
import torch
from torch import nn as nn
from torch import functional as F
import cv2 as cv
import numpy as np
import torchvision
from tqdm import tqdm
from matplotlib import pyplot as plt
import torchinfo

In [2]:
TRAIN_INPUT_DIR = './ISIC2018_Task1-2_Training_Input/'
TRAIN_GT_DIR = './ISIC2018_Task1_Training_GroundTruth/'

VAL_INPUT_DIR = './ISIC2018_Task1-2_Validation_Input/'
VAL_GT_DIR = './ISIC2018_Task1_Validation_GroundTruth/'

BATCH_SIZE = 4
LEARNING_RATE = 1e-5

EPOCHS = 100
IM_H, IM_W = 384, 512

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")

Device: cuda


In [3]:
training_input_files = os.listdir(TRAIN_INPUT_DIR)
for filename in training_input_files:
  if (filename[-3:] != "jpg" and filename[-3:] != "png"):
    training_input_files.remove(filename)
n_train = len(training_input_files)

val_input_files = os.listdir(VAL_INPUT_DIR)
for filename in val_input_files:
  if (filename[-3:] != "jpg" and filename[-3:] != "png"):
    val_input_files.remove(filename)
n_val = len(val_input_files)
n_train -= n_train % BATCH_SIZE
n_val -= n_val % BATCH_SIZE
print(n_train, n_val)

2592 100


In [20]:
def centroid(I):
    p = torch.ones(I.shape).to(I.device.type)
    ipos = torch.cumsum(p, axis=0) - 1
    jpos = torch.cumsum(p, axis=1) - 1
    return (torch.sum(ipos*I)/torch.sum(I)).item() , (torch.sum(jpos*I)/torch.sum(I)).item()

def _centroid(img, lcc=False):
  if lcc:
    img = img.astype(np.uint8)
    nb_components, output, stats, centroids = cv.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    if len(sizes) > 2:
      max_label = 1
      max_size = sizes[1]

      for i in range(2, nb_components):
          if sizes[i] > max_size:
              max_label = i
              max_size = sizes[i]

      img2 = np.zeros(output.shape)
      img2[output == max_label] = 255
      img = img2

  if len(img.shape) > 2:
    M = cv.moments(img[:,:,1])
  else:
    M = cv.moments(img)

  if M["m00"] == 0:
    return (img.shape[0] // 2, img.shape[1] // 2)
  
  cX = int(M["m10"] / M["m00"])
  cY = int(M["m01"] / M["m00"])
  return (cX, cY)

def to_polar(input_img, center):
  input_img = input_img.astype(np.float32)
  value = np.sqrt(((input_img.shape[0]/2.0)**2.0)+((input_img.shape[1]/2.0)**2.0))
  polar_image = cv.linearPolar(input_img, center, value, cv.WARP_FILL_OUTLIERS)
  return polar_image

def to_cart(input_img, center):
  input_img = input_img.astype(np.float32)
  value = np.sqrt(((input_img.shape[1]/2.0)**2.0)+((input_img.shape[0]/2.0)**2.0))
  polar_image = cv.linearPolar(input_img, center, value, cv.WARP_FILL_OUTLIERS + cv.WARP_INVERSE_MAP)
  polar_image = polar_image.astype(np.uint8)
  return polar_image

In [31]:
def torch2np(I):
    J = None
    if (I.dim() == 3):
        J = I.transpose(0, 1).transpose(1, 2).to("cpu").numpy()
    elif (I.dim() == 2):
        J = I.to("cpu").numpy()
    return J

In [30]:
def np2torch(I):
    J = None
    if (I.ndim == 3):
        J = torch.tensor(I).transpose(1, 2).transpose(0, 1)
    elif (I.ndim == 2):
        J = torch.tensor(I)[None,:,:]
    return J

In [5]:
class UNetRes50(nn.Module):
    def __init__(self, device=None, in_channels=None):
        super(UNetRes50, self).__init__()

        if (device is None):
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device
        
        if (in_channels is None):
            self.in_channels = 3
        else:
            self.in_channels = in_channels


        resnet50 = torchvision.models.resnet50(weights="DEFAULT")
        self.conv0 = nn.Conv2d(self.in_channels, 64, (3, 3), stride=(2, 2), padding=1)
        self.e1 = nn.Sequential(self.conv0, resnet50._modules['bn1'], resnet50._modules['relu'])
        self.e2 = nn.Sequential(resnet50._modules['maxpool'], resnet50._modules['layer1'])
        self.e3 = nn.Sequential(resnet50._modules['layer2'])
        self.e4 = nn.Sequential(resnet50._modules['layer3'])
            
        for param in self.e2.parameters():
            param.requires_grad_(False)
            
        for param in self.e3.parameters():
            param.requires_grad_(False)
            
        for param in self.e4.parameters():
            param.requires_grad_(False)
        
        self.c1 = self.convblock(1024, 512)
        self.c2 = self.convblock(512, 256)
        self.c3 = self.convblock(192, 128)
        self.c4 = self.convblock(64+self.in_channels, 64)
        
        self.u1 = nn.ConvTranspose2d(1024, 512, (2, 2), stride=(2, 2))
        self.u2 = nn.ConvTranspose2d(512, 256, (2, 2), stride=(2, 2))
        self.u3 = nn.ConvTranspose2d(256, 128, (2, 2), stride=(2, 2))
        self.u4 = nn.ConvTranspose2d(128, 64, (2, 2), stride=(2, 2))
        
        self.out = nn.Conv2d(64, 1, (1, 1), padding="same")

        self = self.to(self.device)

    def convblock(self, in_features, out_features):
        block = nn.Sequential(
            nn.Conv2d(in_features, out_features, (3, 3), padding="same"),
            nn.BatchNorm2d(out_features),
            nn.ReLU(),
            nn.Conv2d(out_features, out_features, (3, 3), padding="same"),
            nn.BatchNorm2d(out_features),
            nn.ReLU()
            )
        block = block.to(self.device)
        return block


    def forward(self, X):
        X = X.to(self.device)

        s1 = self.e1(X)
        s2 = self.e2(s1)
        s3 = self.e3(s2)
        s4 = self.e4(s3)
        
        s4 = self.u1(s4)
        s3 = self.c1(torch.cat([s3, s4], dim=1))
        s4 = None

        s3 = self.u2(s3)
        s2 = self.c2(torch.cat([s2, s3], dim=1))
        s3 = None

        s2 = self.u3(s2)
        s1 = self.c3(torch.cat([s1, s2], dim=1))
        s2 = None

        s1 = self.u4(s1)
        X = self.c4(torch.cat([X, s1], dim=1))
        s1 = None

        X = torch.sigmoid(self.out(X))

        return X


In [6]:
a = UNetRes50(in_channels=5)
torchinfo.summary(a, input_size=(BATCH_SIZE, 5, IM_H, IM_W))

Layer (type:depth-idx)                        Output Shape              Param #
UNetRes50                                     [4, 1, 384, 512]          --
├─Sequential: 1-1                             [4, 64, 192, 256]         --
│    └─Conv2d: 2-1                            [4, 64, 192, 256]         2,944
│    └─BatchNorm2d: 2-2                       [4, 64, 192, 256]         128
│    └─ReLU: 2-3                              [4, 64, 192, 256]         --
├─Sequential: 1-2                             [4, 256, 96, 128]         --
│    └─MaxPool2d: 2-4                         [4, 64, 96, 128]          --
│    └─Sequential: 2-5                        [4, 256, 96, 128]         --
│    │    └─Bottleneck: 3-1                   [4, 256, 96, 128]         (75,008)
│    │    └─Bottleneck: 3-2                   [4, 256, 96, 128]         (70,400)
│    │    └─Bottleneck: 3-3                   [4, 256, 96, 128]         (70,400)
├─Sequential: 1-3                             [4, 512, 48, 64]          -

In [38]:
class CPCUNet(nn.Module):
    def __init__(self, device=None):
        super(CPCUNet, self).__init__()

        if (device is None):
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device
        
        self.net_0 = UNetRes50(in_channels=3)
        self.net_1 = UNetRes50(in_channels=5)
        self.net_2 = UNetRes50(in_channels=4)
        
        self = self.to(self.device)

    def forward(self, X):
        y_0 = self.net_0(X)
        
        X_polar = torch.zeros((X.shape[0], 5, X.shape[2], X.shape[3])).to(self.device)
        centers = []
        for i in range(X.shape[0]):
            centers.append(centroid(y_0[i,:,:,:]))
            interm = torch2np(y_0[i,:,:,:])
            
            X_polar[i,0:3,:,:] = np2torch(to_polar(interm, centers[i])).to(self.device)
        
        spat = torch.ones((X.shape[0], 1, X.shape[2], X.shape[3])).to(self.device)
        i_spat = (torch.cumsum(spat, dim=2) - 1) / X.shape[2]
        j_spat = (torch.cumsum(spat, dim=3) - 1) / X.shape[3]
        
        X_polar[:,3,:,:] = i_spat
        X_polar[:,4,:,:] = j_spat
        
        y_1 = self.net_1(X_polar)
        
        X_final = torch.zeros((X.shape[0], 4, X.shape[2], X.shape[3]))
        X_final[:,:3,:,:] = X
        
        for i in range(X.shape[0]):
            interm = to_cart(torch2np(y_1[i,:,:,:]), centers[i])
            X_final[i,3:,:,:] = np2torch(interm).to(self.device)
        
        y_2 = self.net_2(X_final)
        
        return y_0, y_1, y_2
            
        
        
        


In [39]:
net = CPCUNet()

In [40]:
torchinfo.summary(net, input_size=(1, 3, IM_H, IM_W))

Layer (type:depth-idx)                             Output Shape              Param #
CPCUNet                                            [1, 1, 384, 512]          --
├─UNetRes50: 1-1                                   [1, 1, 384, 512]          --
│    └─Sequential: 2-1                             [1, 64, 192, 256]         --
│    │    └─Conv2d: 3-1                            [1, 64, 192, 256]         1,792
│    │    └─BatchNorm2d: 3-2                       [1, 64, 192, 256]         128
│    │    └─ReLU: 3-3                              [1, 64, 192, 256]         --
│    └─Sequential: 2-2                             [1, 256, 96, 128]         --
│    │    └─MaxPool2d: 3-4                         [1, 64, 96, 128]          --
│    │    └─Sequential: 3-5                        [1, 256, 96, 128]         (215,808)
│    └─Sequential: 2-3                             [1, 512, 48, 64]          --
│    │    └─Sequential: 3-6                        [1, 512, 48, 64]          (1,219,584)
│    └─Sequenti